In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


ModuleNotFoundError: No module named 'tensorflow.security'

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into 80% training and 20% validation
)

train_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),  # Adjust to the input size expected by ResNet-150
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Four classes

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=4, validation_data=validation_generator)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

test_generator = test_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)


In [ ]:
import tensorflow_model_optimization as tfmot

# Annotate the model for quantization-aware training
quant_annotate_model = tfmot.quantization.keras.quantize_annotate_model(model)


In [ ]:
# Compile the quantization-aware model
quant_annotate_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the quantization-aware model
history = quant_annotate_model.fit(train_generator, epochs=4, validation_data=validation_generator)
